In [14]:
import nltk
from collections import Counter
import pandas as pd

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex

#remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

#20 Function to remove whitespace
def remove_whitespace(text):
    return " ".join(text.split())

# Function to remove punctuations
def remove_punctuations(text):
    words = nltk.word_tokenize(text)
    punt_removed = [w for w in words if w.lower() not in string.punctuation]
    return " ".join(punt_removed)
#30

# Function to remove stop words
def remove_stopwords(text, lang='english'):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

#40
#function for stemming
def words_stemmer(words, type="PorterStemmer", lang="english", encoding="utf8"):
    supported_stemmers = ["PorterStemmer","LancasterStemmer","SnowballStemmer"]
    if type is False or type not in supported_stemmers:
        return words
    else:
        stem_words = []
        if type == "PorterStemmer":
            stemmer = PorterStemmer()
#50
            for word in words:
                stem_words.append(stemmer.stem(word))
        if type == "LancasterStemmer":
            stemmer = LancasterStemmer()
            for word in words:
                stem_words.append(stemmer.stem(word))
        if type == "SnowballStemmer":
            stemmer = SnowballStemmer(lang)
            for word in words:
#60
                stem_words.append(stemmer.stem(word))
    return " ".join(stem_words)



# Function to apply lemmatization to a list of words
def words_lemmatizer(text, encoding="utf8"):
    wordnet_lemmatizer = WordNetLemmatizer()

    words = nltk.word_tokenize(text)
    lemma_words = []
    wl = WordNetLemmatizer()
    for word in words:
        pos = find_pos(word)
        lemma_words.append(wl.lemmatize(word, pos))
    return " ".join(lemma_words)

# Function to find part of speech tag for a word
def find_pos(word):
    # Part of Speech constants
    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
    # You can learn more about these at http://wordnet.princeton.edu/
    #wordnet/man/wndb.5WN.html#sect3
    # You can learn more about all the penn tree tags at https://www.ling.
    #upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
    # Adjective tags - 'JJ', 'JJR', 'JJS'
    if pos.lower()[0] == 'j':
        return 'a'
    # Adverb tags - 'RB', 'RBR', 'RBS'
    elif pos.lower()[0] == 'r':
        return 'r'
    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
    elif pos.lower()[0] == 'v':
        return 'v'
    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
    else:
        return 'n'

# Function to extract n-grams from text
def get_ngrams(text, n):

    n_grams = ngrams(nltk.word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

#load file data into dictionnary 
with open('Microblog_Trialdata.json') as json_data:
    print(type(json_data))
    data_dict = json.load(json_data)
    title0 = data_dict[0]['spans']
    print(type(title0))
    
#print ('Removed numbers:---------------\n ', remove_numbers(text))
#print ('Removed whitespace:--------------- \n ', remove_whitespace(text))
#print ('remove_punctuations---------------- \n', remove_punctuations(text))
#print ('remove_stopwords----------- \n',remove_stopwords(text))
featureList = []
fea_vect=[]
for data in data_dict:
    title0 = data['spans']
    score=data['sentiment score']
    #convert to string
    ch=""
    for w in title0 :
        ch="".join(w)
    #Convert to lower case
    txt_num=remove_numbers(ch)
    txt_ws=remove_whitespace(txt_num)
    txt_pun=remove_punctuations(txt_ws)
    txt_final=remove_stopwords(txt_pun)
    #print ('texte finale --------------\n',txt_final)
    texte=words_lemmatizer(txt_pun)
    n_grams=get_ngrams(texte, 2)
    featureList.extend(n_grams)
    featureList = list(set(featureList))

    #print(Counter(word_tokenize(texte)))
    fea_vect.append((n_grams, score));
    
#fea_vect
print("feature List=",featureList)
with open('ngrams.csv', 'w',newline='') as csvfile:
    filewriter = csv.writer(csvfile,delimiter=' ', quotechar='|')
    for row in fea_vect:
        filewriter.writerow(row[0])


<class '_io.TextIOWrapper'>
<class 'list'>
feature List= ['on dip', 'and be', 'time to', 'rally come', 'far more', 'on a', 'Lunchtime rally', 'Long Position', 'little F', 'the technology', 'it time', 'technology and', 'put up', 'SBUX on', 'Entering long', 'intrigue with', 'GOOGL would', 'picked some', 'Very intrigue', 'Buying SBUX', 'Scaling Up', 'sell bank', 'Market Losers', 'and growth', 'on Long', 'Up on', 'than downside', 'would suck', 'growth potential', 'more upside', 'upside than', 'to sell', 'some up', 'Biggest Market', 'F short', 'be overbought', 'buying opportunity', 'a little', 'Putting on', 'short some', 'with the', 'GOOG GOOGL']


In [2]:
fea_vect

[(['Putting on', 'on a', 'a little', 'little F', 'F short'], -0.454),
 (['short some'], -0.464),
 (['buying opportunity'], 0.445),
 (['Scaling Up', 'Up on', 'on Long', 'Long Position'], 0.661),
 (['it time', 'time to', 'to sell', 'sell bank'], -0.763),
 (['Entering long'], 0.627),
 (['picked some', 'some up'], 0.653),
 (['far more', 'more upside', 'upside than', 'than downside'], 0.668),
 (['Lunchtime rally', 'rally come'], 0.46),
 (['Very intrigue',
   'intrigue with',
   'with the',
   'the technology',
   'technology and',
   'and growth',
   'growth potential'],
  0.403),
 (['put up'], 0.0),
 (['Biggest Market', 'Market Losers'], -0.438),
 (['GOOG GOOGL', 'GOOGL would', 'would suck'], -0.398),
 (['Buying SBUX', 'SBUX on', 'on dip'], 0.483),
 (['and be', 'be overbought'], -0.48),
 (['Putting on', 'on a', 'a little', 'little F', 'F short'], -0.454)]

In [3]:
words=Counter(featureList)

words.most_common()



[('on dip', 1),
 ('and be', 1),
 ('time to', 1),
 ('rally come', 1),
 ('far more', 1),
 ('on a', 1),
 ('Lunchtime rally', 1),
 ('Long Position', 1),
 ('little F', 1),
 ('the technology', 1),
 ('it time', 1),
 ('technology and', 1),
 ('put up', 1),
 ('SBUX on', 1),
 ('Entering long', 1),
 ('intrigue with', 1),
 ('GOOGL would', 1),
 ('picked some', 1),
 ('Very intrigue', 1),
 ('Buying SBUX', 1),
 ('Scaling Up', 1),
 ('sell bank', 1),
 ('Market Losers', 1),
 ('and growth', 1),
 ('on Long', 1),
 ('Up on', 1),
 ('than downside', 1),
 ('would suck', 1),
 ('growth potential', 1),
 ('more upside', 1),
 ('upside than', 1),
 ('to sell', 1),
 ('some up', 1),
 ('Biggest Market', 1),
 ('F short', 1),
 ('be overbought', 1),
 ('buying opportunity', 1),
 ('a little', 1),
 ('Putting on', 1),
 ('short some', 1),
 ('with the', 1),
 ('GOOG GOOGL', 1)]

In [4]:
def build_wordlist(data, min_occurrences=3, max_occurences=500,whitelist=None):
        wordlist = []
        import os
        if os.path.isfile("wordlist_me.csv"):
            word_df = pd.read_csv("wordlist_me.csv")
            word_df = word_df[word_df["occurrences"] > min_occurrences]
            wordlist = list(word_df.loc[:, "word"])
            return

        words = Counter()
        

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_me.csv", index_label="id")
        wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]


In [5]:
df = pd.read_json('Microblog_Trialdata.json')
df

,cashtag,id,sentiment score,source,spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short]
1,$AAPL,10752226,-0.464,stocktwits,[short some]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity]
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position]
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks]
5,$LMT,14073133,0.627,stocktwits,[Entering long]
6,$DNN,18479024,0.653,stocktwits,[picked some up]
7,$CRK,34147106,0.668,stocktwits,"[time to accumulate for a long position, far m..."
8,$CRK,34147106,0.460,stocktwits,"[Looking for a strong bounce, Lunchtime rally ..."
9,$CRK,34147106,0.403,stocktwits,[Very intrigued with the technology and growth...


In [8]:
example_sent = "This is a sample sentence's words, showing off the stop words filtration."

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w in stop_words]

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
ch=" ".join(filtered_sentence)

print(word_tokens)
print(ch)

['This', 'is', 'a', 'sample', 'sentence', "'s", 'words', ',', 'showing', 'off', 'the', 'stop', 'words', 'filtration', '.']
This sample sentence 's words , showing stop words filtration .
